In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import event data
df = pd.read_csv('europaFinal.csv')

df.head()

,minute,second,teamId,x,y,period,type,outcome,playerId,endX,endY
0,0,0.0,Manchester United,0.0,0.0,FirstHalf,Start,Successful,NaN,NaN,NaN
1,0,0.0,Villarreal,0.0,0.0,FirstHalf,Start,Successful,NaN,NaN,NaN
2,0,0.0,Manchester United,49.9,50.0,FirstHalf,Pass,Successful,18.0,31.5,59.6
3,0,3.0,Manchester United,32.2,58.1,FirstHalf,Pass,Unsuccessful,2.0,69.6,100.0
4,0,32.0,Villarreal,34.4,0.0,FirstHalf,Pass,Successful,8.0,36.5,9.5


In [3]:
#Import xT grid
xt = pd.read_csv('xT_Grid.csv', header=None)
xt.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.006383,0.007796,0.008449,0.009777,0.011263,0.012483,0.014736,0.017451,0.021221,0.027563,0.034851,0.037926
1,0.007501,0.008786,0.009424,0.010595,0.012147,0.013845,0.016118,0.018703,0.024015,0.029533,0.040670,0.046477
2,0.008880,0.009777,0.010013,0.011105,0.012692,0.014291,0.016856,0.019351,0.024122,0.028552,0.054911,0.064426
3,0.009411,0.010827,0.010165,0.011324,0.012626,0.014846,0.016895,0.019971,0.023851,0.035113,0.108051,0.257454
4,0.009411,0.010827,0.010165,0.011324,0.012626,0.014846,0.016895,0.019971,0.023851,0.035113,0.108051,0.257454


In [4]:
#Assing values to an array
xt = np.array(xt)

In [5]:
xt_rows, xt_cols = xt.shape

In [6]:
#Bin data for xT
df['x1_bin'] = pd.cut(df['x'], bins=xt_cols, labels=False)
df['y1_bin'] = pd.cut(df['y'], bins=xt_rows, labels=False)
df['x2_bin'] = pd.cut(df['endX'], bins=xt_cols, labels=False)
df['y2_bin'] = pd.cut(df['endY'], bins=xt_rows, labels=False)

In [7]:
#Drop rows that contains NaN values
df.dropna(subset = ['endX'], inplace=True)
df.head()

,minute,second,teamId,x,y,period,type,outcome,playerId,endX,endY,x1_bin,y1_bin,x2_bin,y2_bin
2,0,0.0,Manchester United,49.9,50.0,FirstHalf,Pass,Successful,18.0,31.5,59.6,6,3,3.0,4.0
3,0,3.0,Manchester United,32.2,58.1,FirstHalf,Pass,Unsuccessful,2.0,69.6,100.0,3,4,8.0,7.0
4,0,32.0,Villarreal,34.4,0.0,FirstHalf,Pass,Successful,8.0,36.5,9.5,4,0,4.0,0.0
5,0,34.0,Villarreal,35.1,9.1,FirstHalf,Pass,Successful,7.0,31.9,2.3,4,0,3.0,0.0
8,0,52.0,Villarreal,34.2,0.0,FirstHalf,Pass,Successful,8.0,35.0,10.6,4,0,4.0,0.0


In [10]:
#Convert bin columns to int
df['x2_bin'] = df['x2_bin'].astype(int)
df['y2_bin'] = df['y2_bin'].astype(int)
df.head()

,minute,second,teamId,x,y,period,type,outcome,playerId,endX,endY,x1_bin,y1_bin,x2_bin,y2_bin
2,0,0.0,Manchester United,49.9,50.0,FirstHalf,Pass,Successful,18.0,31.5,59.6,6,3,3,4
3,0,3.0,Manchester United,32.2,58.1,FirstHalf,Pass,Unsuccessful,2.0,69.6,100.0,3,4,8,7
4,0,32.0,Villarreal,34.4,0.0,FirstHalf,Pass,Successful,8.0,36.5,9.5,4,0,4,0
5,0,34.0,Villarreal,35.1,9.1,FirstHalf,Pass,Successful,7.0,31.9,2.3,4,0,3,0
8,0,52.0,Villarreal,34.2,0.0,FirstHalf,Pass,Successful,8.0,35.0,10.6,4,0,4,0


In [11]:
#Implement lambda function to fit bin values (xT Grid) into x and y values 
df['start_zone_value'] = df[['x1_bin', 'y1_bin']].apply(lambda x: xt[x[1]][x[0]], axis=1)
df['end_zone_value'] = df[['x2_bin', 'y2_bin']].apply(lambda x: xt[x[1]][x[0]], axis=1)

In [12]:
#Calculate xT value
df['xT'] = df['end_zone_value'] - df['start_zone_value']

In [13]:
#Observe Bruno Fernandes's xT
df = df.loc[(df['teamId']=='Manchester United') & (df['playerId']==18) & (df['outcome']=='Successful') & (df['type']=='Pass')]
df.head(10)

,minute,second,teamId,x,y,period,type,outcome,playerId,endX,endY,x1_bin,y1_bin,x2_bin,y2_bin,start_zone_value,end_zone_value,xT
2,0,0.0,Manchester United,49.9,50.0,FirstHalf,Pass,Successful,18.0,31.5,59.6,6,3,3,4,0.016895,0.011324,-0.005572
119,6,11.0,Manchester United,65.4,96.8,FirstHalf,Pass,Successful,18.0,63.3,87.8,7,7,7,7,0.017451,0.017451,0.000000
130,6,55.0,Manchester United,69.8,30.3,FirstHalf,Pass,Successful,18.0,74.4,18.8,8,2,8,1,0.024122,0.024015,-0.000107
133,7,1.0,Manchester United,84.3,23.3,FirstHalf,Pass,Successful,18.0,86.0,20.0,10,1,10,1,0.040670,0.040670,0.000000
185,12,20.0,Manchester United,39.0,64.8,FirstHalf,Pass,Successful,18.0,32.8,72.9,4,5,3,5,0.012692,0.011105,-0.001587
197,12,50.0,Manchester United,43.0,70.0,FirstHalf,Pass,Successful,18.0,41.0,96.9,5,5,4,7,0.014291,0.011263,-0.003029
286,19,9.0,Manchester United,69.5,74.5,FirstHalf,Pass,Successful,18.0,59.9,71.1,8,5,7,5,0.024122,0.019351,-0.004771
336,22,57.0,Manchester United,37.0,89.6,FirstHalf,Pass,Successful,18.0,32.8,81.4,4,7,3,6,0.011263,0.010595,-0.000668
366,24,43.0,Manchester United,71.7,24.6,FirstHalf,Pass,Successful,18.0,70.9,9.7,8,1,8,0,0.024015,0.021221,-0.002794
434,30,5.0,Manchester United,49.9,50.1,FirstHalf,Pass,Successful,18.0,31.0,40.7,6,4,3,3,0.016895,0.011324,-0.005572


In [14]:
#Calculate Bruno's total xT for the match
df['xT'].sum()

0.05566478999999999